## Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np 

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
0,0,Lorengau,PG,-2.0226,147.2712,81.86,76,100,8.50,overcast clouds
1,1,Hamilton,US,39.1834,-84.5333,48.72,51,0,4.00,clear sky
2,2,Torbay,CA,47.6666,-52.7314,19.76,72,20,5.75,few clouds
3,3,Kavieng,PG,-2.5744,150.7967,82.72,75,95,9.55,overcast clouds
4,4,Nusaybin,TR,37.0778,41.2178,51.39,93,75,13.80,mist


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temp you would like for your trip?"))
max_temp = float(input("What is the max temp you would like for your trip?"))

What is the min temp you would like for your trip?60
What is the max temp you would like for your trip?75


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp)& (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Desc
12,12,Port Alfred,ZA,-33.5906,26.8910,71.02,81,98,16.49,overcast clouds
21,21,Hwange,ZW,-18.3693,26.5019,67.96,97,100,2.44,light rain
22,22,Ribeira Grande,PT,38.5167,-28.7000,61.20,77,20,14.97,few clouds
25,25,Mareeba,AU,-17.0000,145.4333,73.44,92,92,1.32,overcast clouds
37,37,Cape Town,ZA,-33.9258,18.4232,70.11,66,0,3.00,clear sky


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID         0
City            0
Country         3
Lat             0
Lng             0
Max Temp        0
Humidity        0
Cloudiness      0
Wind Speed      0
Weather Desc    0
dtype: int64

In [6]:
# Drop emmpty rows
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID         176
City            176
Country         176
Lat             176
Lng             176
Max Temp        176
Humidity        176
Cloudiness      176
Wind Speed      176
Weather Desc    176
dtype: int64

In [7]:
# reate DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Desc", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
12,Port Alfred,ZA,71.02,overcast clouds,-33.5906,26.8910,
21,Hwange,ZW,67.96,light rain,-18.3693,26.5019,
22,Ribeira Grande,PT,61.20,few clouds,38.5167,-28.7000,
25,Mareeba,AU,73.44,overcast clouds,-17.0000,145.4333,
37,Cape Town,ZA,70.11,clear sky,-33.9258,18.4232,
40,Busselton,AU,63.14,few clouds,-33.6500,115.3333,
41,Port Shepstone,ZA,72.21,overcast clouds,-30.7414,30.4550,
43,East London,ZA,71.51,overcast clouds,-33.0153,27.9116,
47,Carnarvon,AU,73.47,clear sky,-24.8667,113.6333,
52,Bredasdorp,ZA,67.32,overcast clouds,-34.5322,20.0403,


In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]= f"{lat}, {lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params)
    hotels = hotels.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"]= hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
# Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace= True)
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head()

,City,Country,Max Temp,Weather Desc,Lat,Lng,Hotel Name
12,Port Alfred,ZA,71.02,overcast clouds,-33.5906,26.8910,The Halyards Hotel
21,Hwange,ZW,67.96,light rain,-18.3693,26.5019,Hwange Colliery Guest House
22,Ribeira Grande,PT,61.20,few clouds,38.5167,-28.7000,Quinta da Meia Eira
25,Mareeba,AU,73.44,overcast clouds,-17.0000,145.4333,Trinity Plains Tourist Park
37,Cape Town,ZA,70.11,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town


In [10]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. add to template box: city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure
fig


Figure(layout=FigureLayout(height='420px'))